In [3]:
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
from typing import TypedDict
from dotenv import load_dotenv

In [4]:
# creating the state 
class ai_monitoring_state(TypedDict):
    query:str
    context:str
    llm_answer:str
    confidence:int
    risk_level:str
    decision:str
    sources:str

In [ ]:
# Loading the user document 
